In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# activate R magic
%load_ext rpy2.ipython

In [3]:
cd 'drive/My Drive/Portfolio/online-learning-ab-testing/data/'

/content/drive/My Drive/Portfolio/online-learning-ab-testing/data


In [4]:
ls

 data_cleaned.csv        Participant.csv        Videos.csv
'Data Dictionary.xlsx'   ParticipantVideo.csv


In [5]:
# downgrade to avoid problems
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
%%R
getwd()

[1] "/content/drive/My Drive/Portfolio/online-learning-ab-testing/data"


In [7]:
%%R
install.packages("data.table")
library(data.table)
install.packages("stargazer")
library(stargazer)
install.packages("plm")
library(plm)
install.packages("AER")
library(AER)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpM0Lo4N/downloaded_packages’


(as ‘lib’ is unspecified)







	‘/tmp/RtmpM0Lo4N/downloaded_packages’

Please cite as: 





(as ‘lib’ is unspecified)
































































	‘/tmp/RtmpM0Lo4N/downloaded_packages’

Attaching package: ‘plm’



    between


(as ‘lib’ is unspecified)





































































	‘/tmp/RtmpM0Lo4N/downloaded_packages’





Attaching package: ‘zoo’



    as.Date, as.Date.numeric






In [8]:
%%R
#load dataset 
MyData <- fread("data_cleaned.csv", verbose = F)

## 1. Intention to Treat (ITT)

Does the requirement to use their camera have a causal effect on a participant’s learning?

In [9]:
%%R 
itt <- lm(correct_post_quiz ~
                      recording_elegible +
                      correct_pre_quiz +
                      video_id +
                      survey_duration +
                      factor(demo_gender) +
                      factor(demo_race),
                    data = MyData)

## 2. Local Average Treatment Effect (LATE)

Does the decision to turn on their camera have a causal effect on a participant’s learning?

In [10]:
%%R
late <- ivreg(correct_post_quiz ~
                      cam_allowed +
                      correct_pre_quiz +
                      video_id +
                      survey_duration +
                      factor(demo_gender) +
                      factor(demo_race)
                      |recording_elegible +
                      correct_pre_quiz +
                      video_id +
                      survey_duration +
                      factor(demo_gender) +
                      factor(demo_race),
                    data = MyData)

## 3. LATE within LATE

Does having camera on for at least 40% of the video duration have a causal effect on a participant’s learning?

In [11]:
%%R
late2 <- ivreg(correct_post_quiz ~
                      well_recorded +
                      correct_pre_quiz +
                      video_id +
                      survey_duration +
                      factor(demo_gender) +
                      factor(demo_race)
                      |cam_allowed + # explain cam_allowed is exogenous (there are individual effects, IV should be able to explain some variance in well_recorded; 
                                      # this variable may be correlated with post quiz score); discuss correlation between IV and outcome, impact on causal effects
                      correct_pre_quiz +
                      video_id +
                      survey_duration +
                      factor(demo_gender) +
                      factor(demo_race),
                    data = MyData)

## 4. LATE with Heterogeneous Effect

Is there any different causal effect for the groups with and without prior knowledge on the concept taught?


In [12]:
%%R
lateth <- ivreg(correct_post_quiz
                          ~ well_recorded +
                            pre_quiz_all_correct +
                            well_recorded:pre_quiz_all_correct +
                            correct_pre_quiz +
                            video_id +
                            survey_duration +
                            factor(demo_gender) +
                            factor(demo_race)
                          |cam_allowed +
                            pre_quiz_all_correct +
                            cam_allowed:pre_quiz_all_correct +
                            correct_pre_quiz +
                            video_id +
                            survey_duration +
                            factor(demo_gender) +
                            factor(demo_race),
                          data = MyData)

## Outcome

In [13]:
%%R
stargazer(
  itt,
  late,
  late2,
  lateth,
  title="Impact of participant recording on learning outcome",
  column.labels=c("ITT", "LATE Allow Camera", "LATE Well Recorded",
                  "LATE Well Recorded Heterogeneous Effects"),
  omit = c('correct\\_pre\\_quiz', 'video\\_id', 'survey\\_duration',
           'demo\\_gender', 'demo\\_race'),
  omit.labels = c('correct\\_pre\\_quiz', 'video\\_id', 'survey\\_duration',
           'demo\\_gender', 'demo\\_race'),
  type = "text",
  model.numbers = F)


Impact of participant recording on learning outcome
                                                                            Dependent variable:                                         
                                   -----------------------------------------------------------------------------------------------------
                                                                             correct_post_quiz                                          
                                             OLS                                           instrumental                                 
                                                                                             variable                                   
                                             ITT           LATE Allow Camera LATE Well Recorded LATE Well Recorded Heterogeneous Effects
-----------------------------------------------------------------------------------------------------------------------------